In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.stats import zscore
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
data2 = '../../data_sets/5 yrs NBME Scores_4-2020_3-2024.xlsx'
data_CK = '../../data_sets/STEP2CK2019-2023.xlsx'
data_CBSE = '../../data_sets/5 yrs CBSE Scores_1-2020_1-2024.xlsx'
data_CCSE = '../../data_sets/5 yrs CCSE Scores_1-2020_1-2024.xlsx'

ck_id = '../../data_sets/IDs.xlsx'
pd.set_option('display.max_columns', None)

In [3]:
clerkships =  pd.concat(pd.read_excel(data2, sheet_name=None),axis =0, ignore_index=True)

df_f = pd.read_excel(data2,sheet_name = 0)
df_i = pd.read_excel(data2,sheet_name = 1)
df_n = pd.read_excel(data2,sheet_name = 2)
df_o = pd.read_excel(data2,sheet_name = 3)
df_pd = pd.read_excel(data2,sheet_name = 4)
df_ps = pd.read_excel(data2,sheet_name = 5)
df_s = pd.read_excel(data2,sheet_name = 6)

ck = pd.read_excel(ck_id)
ck_scores = pd.read_excel(data_CK)
cbse_scores = pd.read_excel(data_CBSE)
ccse_scores = pd.read_excel(data_CCSE)

In [4]:
cbse_scores = cbse_scores[['ID','Total Test',]]

In [5]:
ccse_scores= ccse_scores[['ID', 'Total Test',]]

In [6]:
ck_scores = ck_scores.merge(ck, how="inner", left_on='Usmle Id',right_on= "USMLE ID")
ck_scores.rename(columns={'Banner ID_x':'ID'}, inplace=True)


In [7]:
result1 = df_f.merge(df_i, how= 'inner', on=['ID'])
result2 = df_n.merge(df_o, how= 'inner', on=['ID'])
result3 = df_pd.merge(df_ps, how= 'inner', on=['ID'])
clerkship_data = result1.merge(result2, how= 'inner', on=['ID'])
clerkship_data = clerkship_data.merge(result3, how= 'inner', on=['ID'])
clerkship_data = clerkship_data.merge(df_s, how= 'inner', on=['ID'])

print(clerkship_data.shape)

#clerkship_data = clerkship_data.merge(cbse_scores, how= 'inner', on=['ID'])
print(clerkship_data.shape)

clerkship_data = clerkship_data.merge(ccse_scores, how= 'inner', on=['ID'])
print(clerkship_data.shape)

clerkship_data = clerkship_data.merge(ck_scores, how="inner", left_on='ID',right_on= "Banner ID")
print(clerkship_data.shape)

clerkship_data.rename(columns={'Total Test':'CCSE_score'}, inplace=True)
#'Total Test_x':'CBSE_score',

clerkship_data =clerkship_data[['FAM 7100 Attempt# 1','INT 7200 Attempt# 1','NEU 7700 Attempt# 1',
                  'OBS 7600 Attempt# 1','PED 7300 Attempt# 1','PSY 7500 Attempt# 1','SUR 7400 Attempt# 1','CCSE_score','3-dgt score']]
#,'CBSE_score'
print(clerkship_data.shape)


(1222, 47)
(1222, 47)
(892, 48)
(913, 62)
(913, 9)


In [8]:
clerkships.loc[clerkships.Exam == 'Family Medicine Modular Core + Chronic + Musculoskeletal', 'class'] = "FAM 7100"
clerkships.loc[clerkships.Exam == 'Medicine', 'class'] = "INT 7200"
clerkships.loc[clerkships.Exam == 'Clinical Neurology', 'class'] = "NEU 7700"
clerkships.loc[clerkships.Exam == 'Obstetrics and Gynecology', 'class'] = "OBS 7600"
clerkships.loc[clerkships.Exam == 'Pediatrics', 'class'] = "PED 7300"
clerkships.loc[clerkships.Exam == 'Psychiatry', 'class'] = "PSY 7500"
clerkships.loc[clerkships.Exam == 'Surgery', 'class'] = "SUR 7400"

clerkships.loc[clerkships.Exam == 'Family Medicine Modular Core + Chronic + Musculoskeletal', 'gmp'] = 59
clerkships.loc[clerkships.Exam == 'Medicine', 'gmp'] = 58
clerkships.loc[clerkships.Exam == 'Clinical Neurology', 'gmp'] = 61
clerkships.loc[clerkships.Exam == 'Obstetrics and Gynecology', 'gmp'] = 61
clerkships.loc[clerkships.Exam == 'Pediatrics', 'gmp'] = 60
clerkships.loc[clerkships.Exam == 'Psychiatry', 'gmp'] = 69
clerkships.loc[clerkships.Exam == 'Surgery', 'gmp'] = 57

In [9]:
clerkships_dash = clerkships
clerkships_dash["percent_rank"] = clerkships_dash.groupby("class")["Attempt# 1 Score"].transform("rank", pct=True)
clerkships_dash["P/F"] = np.where(clerkships_dash['Attempt# 1 Score'] >= clerkships_dash['gmp'] ,"Pass","Fail")

In [10]:
clerkships_dash = clerkships[['Test Date(s)', 'Exam', 'ID','Examinee', 'Attempt# 1 Score', 'class', 'gmp','percent_rank','P/F']]

In [11]:
clerkships_dash.shape

(8205, 9)

In [12]:
clerkships_dash.head(2)

,Test Date(s),Exam,ID,Examinee,Attempt# 1 Score,class,gmp,percent_rank,P/F
0,2020-04-30,Family Medicine Modular Core + Chronic + Muscu...,4316134,BULAK KAROLINA,72,FAM 7100,59.0,0.297145,Pass
1,2020-05-08,Family Medicine Modular Core + Chronic + Muscu...,4598507,ASANTE JENNIFER ASEYE,68,FAM 7100,59.0,0.133218,Pass


In [13]:
clerkship_data.head(2)

,FAM 7100 Attempt# 1,INT 7200 Attempt# 1,NEU 7700 Attempt# 1,OBS 7600 Attempt# 1,PED 7300 Attempt# 1,PSY 7500 Attempt# 1,SUR 7400 Attempt# 1,CCSE_score,3-dgt score
0,70,78,72,77,77,82,73,226,234
1,82,77,81,87,73,87,77,206,249


In [14]:
with pd.ExcelWriter('dataset.xlsx') as writer:
    clerkship_data.to_excel(writer, sheet_name = 'dataset', index = False)
    clerkships_dash.to_excel(writer, sheet_name = 'dashboard', index = False)